In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
try:
    df = pd.read_csv('../data/insurance_data.csv')
    print("Dataset loaded successfully.")
    # Quick cleanup: remove rows where TotalPremium is zero or NaN to avoid division errors
    df = df[df['TotalPremium'] > 0]
except FileNotFoundError:
    print("Error: 'insurance_data.csv' not found. Please run 'dvc pull' first.")

df.head()

C:\Users\Yoga i7\AppData\Local\Temp\ipykernel_17624\3136554232.py:9: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/insurance_data.csv')


Dataset loaded successfully.


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
5,145247,12827,2015-01-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,3.256435,0.0
6,145247,12827,2015-04-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,50.474737,0.0


In [2]:
# 1. Claim Frequency Metric: A binary flag indicating if a claim was made.
df['HasClaim'] = (df['TotalClaims'] > 0).astype(int)

# 2. Margin (Profit) Metric
df['Margin'] = df['TotalPremium'] - df['TotalClaims']

# 3. Claim Severity Data: This metric only applies to policies that HAD a claim.
# We create a separate DataFrame for this analysis.
claims_df = df[df['HasClaim'] == 1].copy()

print(f"Total policies: {len(df)}")
print(f"Policies with claims: {len(claims_df)}")
print(f"Overall Claim Frequency: {df['HasClaim'].mean():.2%}")

Total policies: 618176
Policies with claims: 2641
Overall Claim Frequency: 0.43%


In [3]:
# 1. Create a contingency table (crosstab)
province_claim_freq_table = pd.crosstab(df['Province'], df['HasClaim'])
print("Contingency Table (Frequency):")
print(province_claim_freq_table)

# 2. Perform the Chi-Squared Test
chi2, p_value, dof, expected = stats.chi2_contingency(province_claim_freq_table)

# 3. Analyze and Report
print(f"\n--- Province vs. Claim Frequency ---")
print(f"Chi-Squared Statistic: {chi2:.2f}")
print(f"P-value: {p_value}")

# Interpretation
alpha = 0.05
if p_value < alpha:
    print(f"\nResult: We REJECT the null hypothesis (p < {alpha}).")
    print("Interpretation: There IS a statistically significant association between Province and the likelihood of making a claim.")
    print("Business Recommendation: ACIS should investigate why certain provinces have higher claim frequencies. This could justify regional risk adjustments in premium calculations.")
else:
    print(f"\nResult: We FAIL to reject the null hypothesis (p >= {alpha}).")
    print("Interpretation: There is no statistically significant association between Province and claim frequency.")
    

Contingency Table (Frequency):
HasClaim            0     1
Province                   
Eastern Cape    19647    47
Free State       5923     9
Gauteng        239539  1243
KwaZulu-Natal  111443   453
Limpopo         17943    66
Mpumalanga      31538   125
North West      89465   334
Northern Cape    3635     8
Western Cape    96402   356

--- Province vs. Claim Frequency ---
Chi-Squared Statistic: 93.69
P-value: 8.250192793720945e-17

Result: We REJECT the null hypothesis (p < 0.05).
Interpretation: There IS a statistically significant association between Province and the likelihood of making a claim.
Business Recommendation: ACIS should investigate why certain provinces have higher claim frequencies. This could justify regional risk adjustments in premium calculations.


In [4]:
# 1. Prepare the data for ANOVA
# Get a list of claim amounts for each province
provinces = claims_df['Province'].unique()
province_claim_amounts = [claims_df['TotalClaims'][claims_df['Province'] == p] for p in provinces]

# 2. Perform the ANOVA Test
f_stat, p_value = stats.f_oneway(*province_claim_amounts)

# 3. Analyze and Report
print(f"\n--- Province vs. Claim Severity (ANOVA) ---")
print(f"F-Statistic: {f_stat:.2f}")
print(f"P-value: {p_value}")

# Interpretation
if p_value < alpha:
    print(f"\nResult: We REJECT the null hypothesis (p < {alpha}).")
    print("Interpretation: There IS a statistically significant difference in the average claim amount (severity) among the provinces.")
    print("Business Recommendation: Premiums should not only account for claim frequency but also the expected cost of claims, which varies by region. High-severity provinces may require higher premiums.")
else:
    print(f"\nResult: We FAIL to reject the null hypothesis (p >= {alpha}).")
    print("Interpretation: We cannot conclude that the average claim amount differs significantly across provinces.")


--- Province vs. Claim Severity (ANOVA) ---
F-Statistic: 4.81
P-value: 6.7238630263519155e-06

Result: We REJECT the null hypothesis (p < 0.05).
Interpretation: There IS a statistically significant difference in the average claim amount (severity) among the provinces.
Business Recommendation: Premiums should not only account for claim frequency but also the expected cost of claims, which varies by region. High-severity provinces may require higher premiums.


In [5]:
# --- ZIP CODE ANALYSIS ---
# 1. Find the top 5 most frequent postal codes
top_5_zipcodes = df['PostalCode'].value_counts().nlargest(5).index
df_top_zips = df[df['PostalCode'].isin(top_5_zipcodes)].copy()
claims_df_top_zips = claims_df[claims_df['PostalCode'].isin(top_5_zipcodes)].copy()

print(f"\n--- Analyzing Top 5 Zip Codes: {list(top_5_zipcodes)} ---")

# A. Risk - Frequency (Chi-Squared)
zip_freq_table = pd.crosstab(df_top_zips['PostalCode'], df_top_zips['HasClaim'])
chi2, p_val_freq, _, _ = stats.chi2_contingency(zip_freq_table)
print(f"\nZip Code vs Claim Frequency P-value: {p_val_freq}")
if p_val_freq < alpha:
    print("Result: REJECT H₀. Claim frequency differs significantly across these top zip codes.")
else:
    print("Result: FAIL to reject H₀. No significant difference in claim frequency.")

# B. Risk - Severity (ANOVA)
zip_claim_amounts = [claims_df_top_zips['TotalClaims'][claims_df_top_zips['PostalCode'] == zc] for zc in top_5_zipcodes]
f_stat, p_val_sev, = stats.f_oneway(*zip_claim_amounts)
print(f"\nZip Code vs Claim Severity P-value: {p_val_sev}")
if p_val_sev < alpha:
    print("Result: REJECT H₀. Claim severity differs significantly across these top zip codes.")
else:
    print("Result: FAIL to reject H₀. No significant difference in claim severity.")

# C. Margin (ANOVA)
zip_margins = [df_top_zips['Margin'][df_top_zips['PostalCode'] == zc] for zc in top_5_zipcodes]
f_stat, p_val_margin, = stats.f_oneway(*zip_margins)
print(f"\nZip Code vs Margin P-value: {p_val_margin}")
if p_val_margin < alpha:
    print("Result: REJECT H₀. Margin (profitability) differs significantly across these top zip codes.")
    print("Business Recommendation: Hyper-local pricing strategies could be very effective. ACIS can identify profitable zip codes for targeted marketing and adjust premiums in less profitable ones.")
else:
    print("Result: FAIL to reject H₀. No significant difference in margin.")


--- Analyzing Top 5 Zip Codes: [2000, 122, 299, 7784, 7405] ---

Zip Code vs Claim Frequency P-value: 1.5125585999138908e-08
Result: REJECT H₀. Claim frequency differs significantly across these top zip codes.

Zip Code vs Claim Severity P-value: 0.0014151202645841159
Result: REJECT H₀. Claim severity differs significantly across these top zip codes.

Zip Code vs Margin P-value: 0.09577865859994268
Result: FAIL to reject H₀. No significant difference in margin.


In [6]:
# --- GENDER ANALYSIS ---
# 1. Clean the 'Gender' column - we only want 'Male' and 'Female' for a clean test
df_gender = df[df['Gender'].isin(['Male', 'Female'])].copy()
claims_df_gender = claims_df[claims_df['Gender'].isin(['Male', 'Female'])].copy()

# A. Risk - Frequency (Chi-Squared)
gender_freq_table = pd.crosstab(df_gender['Gender'], df_gender['HasClaim'])
print("\nGender Contingency Table:")
print(gender_freq_table)
chi2, p_val_freq_gender, _, _ = stats.chi2_contingency(gender_freq_table)
print(f"\nGender vs Claim Frequency P-value: {p_val_freq_gender}")
if p_val_freq_gender < alpha:
    print("Result: REJECT H₀. There is a significant association between gender and claim frequency.")
else:
    print("Result: FAIL to reject H₀.")


# B. Risk - Severity (T-Test)
# Question: Is the average claim amount different for men vs. women?
# Variables: TotalClaims (Numerical) vs. Gender (Categorical, 2 groups).
# Correct Test: Independent Samples T-Test.

male_claims = claims_df_gender[claims_df_gender['Gender'] == 'Male']['TotalClaims']
female_claims = claims_df_gender[claims_df_gender['Gender'] == 'Female']['TotalClaims']

# Perform the T-Test. Use equal_var=False as it's more robust.
t_stat, p_val_sev_gender = stats.ttest_ind(male_claims, female_claims, equal_var=False, nan_policy='omit')

print(f"\nGender vs Claim Severity P-value: {p_val_sev_gender}")
if p_val_sev_gender < alpha:
    print("Result: REJECT H₀. There is a significant difference in claim severity between men and women.")
    # Find out who has higher claims
    if male_claims.mean() > female_claims.mean():
        print("Insight: On average, men's claims are more severe (costly) than women's.")
    else:
        print("Insight: On average, women's claims are more severe (costly) than men's.")
    print("Business Recommendation: Gender can be a significant rating factor if regulations permit. The difference in claim cost should be reflected in premium calculations.")
else:
    print("Result: FAIL to reject H₀.")


Gender Contingency Table:
HasClaim      0   1
Gender             
Female     3391  13
Male      18998  85

Gender vs Claim Frequency P-value: 0.7061410625096163
Result: FAIL to reject H₀.

Gender vs Claim Severity P-value: 0.43113554778854557
Result: FAIL to reject H₀.
